# Import and Tidy 
---------------------------------------------------------------------------------------------------

The purpose of this notebook is to import data from provided data sources and transform it into the Haystax's standard format. This involves cleaning, and tidying resulting in tidy data that has a consistent format and can be used for exploratory data analysis.

## Set Environments
Let's first install some packages (python and R) that we shall use for our analysis. We shall also set up our plotting requirements.

In [2]:
%matplotlib inline
import numpy as np
import pandas as pd
import dask.dataframe as dd
import seaborn as sns
import matplotlib.pyplot as plt
sns.set_context('notebook', font_scale = 1.1)
np.random.seed(12345)
rc = {'xtick.labelsize': 40, 'ytick.labelsize': 40, 'axes.labelsize': 40, 'font.size': 40, 'lines.linewidth': 4.0, 
      'lines.markersize': 40, 'font.family': "serif", 'font.serif': "cm", 'savefig.dpi': 200,
      'text.usetex': False, 'legend.fontsize': 40.0, 'axes.titlesize': 40, "figure.figsize": [24, 16]}
sns.set(rc = rc)
sns.set_style("darkgrid")
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import os

## Set Data Sources

Import data from the provided data sources such as:
* Flat files e.g csv, json, text files
* Databases e.g NoSQL (MongoDB), RDBMS (MS SQL)
* Distributed File Systems e.g. Hadoop,
* etc.

The `pydata` stack provides several packages to read data these data sources. We shall implement these with each clients requirements.

In [3]:
# First, specify data source here
hard_disk = "/opt/usb"
folder = "cert/standardized"
file_type = "*.parquet" # may include csv, hdf, json, text, etc

raw_data_email = os.path.join(hard_disk, "cert/r6.2/email.csv")
raw_data_hr = os.path.join(hard_disk, "cert/r6.2/hr.csv")
raw_data_print = os.path.join(hard_disk, "cert/r6.2/print.csv")
raw_data_badge = os.path.join(hard_disk, "cert/r6.2/badge.csv")
raw_data_network = os.path.join(hard_disk, "cert/r6.2/logon.csv")
raw_data_usb = os.path.join(hard_disk, "cert/r6.2/device.csv")

working_dir = os.path.join(hard_disk, folder)

## Email Data
-----
### Import Raw Email Data
If the data is a flat file, let's read in the data using `dask`.

In [9]:
# for csv files
ddf = dd.read_csv(raw_data_email, parse_dates=["date"])

### Tidy the Email Data
Let's clean the data and tidy it into a constient semantic.  
Let's view the first three records and the structure of the resulting `pandas` dataframe.  

In [57]:
df = ddf.head(n = 3)
df

,id,date,user,pc,to,cc,bcc,from,activity,size,attachments,content
0,{I1O2-B4EB49RW-7379WSQW},2010-01-02 06:36:41,HDB1666,PC-6793,Louis.Bernard.Garza@dtaa.com,Emery.Ali.Holloway@dtaa.com,Hector.Donovan.Bray@dtaa.com,Hector.Donovan.Bray@dtaa.com,Send,45659,NaN,"Now Sylvia, the object of Aminta's desire, arr..."
1,{L7E7-V4UX89RR-3036ZDHU},2010-01-02 06:40:02,HDB1666,PC-6793,Hector.Donovan.Bray@dtaa.com,NaN,NaN,Luke.Grant.Mcmahon@dtaa.com,View,34142,NaN,"On May 14, they picked up 44 more Iroquois at ..."
2,{S8C2-Q8YX87DJ-0516SIWZ},2010-01-02 06:42:48,HDB1666,PC-6793,Quintessa.O.Farrell@harris.com,Hector.Donovan.Bray@dtaa.com,NaN,Hector.Donovan.Bray@dtaa.com,Send,1310925,C:\28X79b6\0PAGXTJ8.doc(1119253);C:\11b38g6\5M...,Sylvia is notable for its mythological Arcadia...


Our `pandas` dataframe comprises columns that we are interested in such as "user" (username), "date", "to" (the recipient email address) and "size" (attachment size) of emails.

In [58]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 12 columns):
id             3 non-null object
date           3 non-null datetime64[ns]
user           3 non-null object
pc             3 non-null object
to             3 non-null object
cc             2 non-null object
bcc            1 non-null object
from           3 non-null object
activity       3 non-null object
size           3 non-null int64
attachments    1 non-null object
content        3 non-null object
dtypes: datetime64[ns](1), int64(1), object(10)
memory usage: 368.0+ bytes


In [59]:
def standardize_columns_email(ddf, working_dir):
    
    required_columns = ["record_id", "sender_employee_id", "sender_username", 
                        "subject", "timestamp", "number_of_attachments", 
                        "attachment_size", "email_text", "file_date"]

    ddf = ddf.rename(columns = {"id" : "record_id",
                        "user" : "sender_employee_id",
                        "from" : "sender_username",
                        "date" : "timestamp",
                         "attachments" : "number_of_attachments",
                        "size" : "attachment_size", 
                        "content": "email_text"
                        })

    ddf["subject"] = ""
    ddf["file_date"] = ""
    ddf = ddf[required_columns]
    
    ddf[["subject", "email_text"]] = ddf[["subject", "email_text"]].astype('str')
    ddf["number_of_attachments"] = ddf["number_of_attachments"].astype('int64')
    ddf["attachment_size"] = ddf["attachment_size"].astype('float64')
    ddf = ddf.categorize(columns = ["record_id", "sender_employee_id", "sender_username"])
#     ddf[["record_id", "sender_employee_id", 
#          "sender_username"]] = ddf[["record_id", "sender_employee_id", 
#                                     "sender_username"]].astype('category')
    
    # save the standardized data as a distributed parquet file
    ddf.to_parquet(path = working_dir)

In [60]:
standardize_columns_email(ddf = ddf, working_dir = working_dir+"/email")

In [28]:
dd.read_parquet(working_dir+"/email/*.parquet").head(n = 1).info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 9 columns):
record_id                1 non-null object
sender_employee_id       1 non-null object
sender_username          1 non-null object
subject                  1 non-null object
timestamp                1 non-null datetime64[ns]
number_of_attachments    0 non-null object
attachment_size          1 non-null int64
email_text               1 non-null object
file_date                1 non-null object
dtypes: datetime64[ns](1), int64(1), object(7)
memory usage: 152.0+ bytes


## USB Data
----
### Import USB Data
USB data is provided by the `device.csv` file in CERT.  
`device.csv` shows the connection and disconnet status of each employee that plugged in a USB. 

Users are assigned a normal/average number of thumb drive uses per day. Deviations from a user's normal usage can be considered significant.

In [5]:
# for csv files
ddf = dd.read_csv(raw_data_usb, parse_dates=["date"])

### Tidy the USB Data
Let's clean the data and tidy it into a constient semantic.  
Let's view the first three records and the structure of the resulting `pandas` dataframe.  

In [6]:
df = ddf.head(n = 3)
df

,id,date,user,pc,file_tree,activity
0,{Z2Q8-K3AV28BE-9353JIRT},2010-01-02 07:17:18,SDH2394,PC-5849,R:\;R:\22B5gX4;R:\SDH2394,Connect
1,{C7F1-G7LE60RU-2483DAXS},2010-01-02 07:22:42,JKS2444,PC-6961,R:\;R:\JKS2444,Connect
2,{T9A4-D4RV69OF-1704NINW},2010-01-02 07:31:42,CBA1023,PC-1570,R:\;R:\42gY283;R:\48rr4y2;R:\59ntt61;R:\76xCQG...,Connect


Our `pandas` dataframe comprises columns that we are interested in such as "user" (username), "date", "to" (the recipient email address) and "size" (attachment size) of emails.

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 6 columns):
id           3 non-null object
date         3 non-null datetime64[ns]
user         3 non-null object
pc           3 non-null object
file_tree    3 non-null object
activity     3 non-null object
dtypes: datetime64[ns](1), object(5)
memory usage: 224.0+ bytes


In [8]:
def standardize_columns_usb(ddf, working_dir):
    
    required_columns = ["record_id", "employee_id", "timestamp", "computer_id", 
                        "file_tree", "connect_activity"]

    ddf = ddf.rename(columns = {"id" : "record_id",
                        "user" : "employee_id",
                        "date" : "timestamp",
                         "pc" : "computer_id", 
                        "activity": "connect_activity"
                        })

    ddf = ddf[required_columns]
    
    ddf["file_tree"] = ddf["file_tree"].astype('str')
    ddf = ddf.categorize(columns = ["record_id", "employee_id", 
                                    "computer_id", "connect_activity"])
#     ddf[["record_id", "employee_id", "computer_id" 
#          "connect_activity"]] = ddf[["record_id", "employee_id", "computer_id", 
#                                     "connect_activity"]].astype('category')
    
    # save the standardized data as a distributed parquet file
    ddf.to_parquet(path = working_dir)

In [9]:
standardize_columns_usb(ddf, working_dir+"/usb")

Verify the final data structure.

In [10]:
dd.read_parquet(working_dir+"/usb/*.parquet").head(n = 1).info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 6 columns):
record_id           1 non-null object
employee_id         1 non-null object
timestamp           1 non-null datetime64[ns]
computer_id         1 non-null object
file_tree           1 non-null object
connect_activity    1 non-null object
dtypes: datetime64[ns](1), object(5)
memory usage: 128.0+ bytes


## Print Data
### Import Raw Print Data

In [ ]:
# for csv files
ddf = dd.read_csv(raw_data_print, parse_dates=["date"])

## Network Data
----
### Import Raw Network Data
Network data is provided by the `logon.csv` file in CERT.  
`logon.csv` shows logon/logoff activity status of employees to the company network.

In [ ]:
# for csv files
ddf = dd.read_csv(raw_data_network, parse_dates=["date"])

# Final thoughts

This study proposed a Bayesian nonparametric framework to capture implicitly hidden structure in time-series having limited data. The proposed framework, a Gaussian process with a spectral mixture kernel, was applied to time-series process for insider-threat data. The proposed framework addresses two current challenges when analyzing quite noisy time-series having limited data whereby the time series are visualized for noticeable structure such as periodicity, growing or decreasing trends and hard coding them into pre-specified functional forms. Experiments demonstrated that results from this framework outperform traditional ARIMA when the time series does not have easily noticeable structure and is quite noisy. Future work will involve evaluating the proposed framework on other different types of insider-threat behavior.

# Computing Environment

The following computing environment was used to generate the above analysis.

In [ ]:
# print system information/setup
%reload_ext watermark
%watermark -v -m -p numpy,pandas,matplotlib,ipywidgets,seaborn -g

In [ ]:
os.environ